In [0]:
%pyspark

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

habrData = spark.read.option("header", True)\
.option("inferSchema", True)\
.csv("/user/admin/habr_data.csv")\
.withColumn("rating", col("rating").cast(IntegerType()))\
.cache()

habrData.printSchema()
habrData.show()

In [1]:
z.show(
    habrData
    )

По данным habr_data получить таблицу с названиями топ-3 статей (по rating) для каждого автора.

In [3]:
%pyspark
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

df = habrData
dfWindow = Window.partitionBy('author_name').orderBy(col('rating').desc())

df2 = df.select('author_name', 'title', 'rating').\
withColumn('row_number', row_number().over(dfWindow)).\
where('row_number < 4')

z.show(df2)

По данным habr_data получить топ (по встречаемости) английских слов из заголовков.
Возможное решение: 1) выделение слов с помощью регулярных выражений, 2) разделение на массивы слов 3) explode массивовов 4) группировка с подсчетом встречаемости


In [5]:
%pyspark

from pyspark.sql.functions import regexp_replace, udf, explode, split
from pyspark.sql.types import StringType, ArrayType

hd = habrData.select('title').\
withColumn('title2', regexp_replace(col('title'), '[^a-zA-Z\s]', ' ')).\
withColumn('title3', regexp_replace(col('title2'), '\s{2,}', ' '))

hd = hd.select('title', 'title3').\
withColumn('title4', split(col('title3'), ' '))

hd = hd.select(explode(hd.title4).alias('words')).\
na.drop('all').\
groupBy('words').count().orderBy(col('count').desc())

hd.printSchema()

z.show(hd)